In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
from data.data_source import get_data_source
from data.treasury_curve import get_yield_curve
from models.pricing_models.bond_model import Bond

ds = get_data_source()

def run_valuation(asof_str):
    asof = pd.to_datetime(asof_str)

    # load inventory
    inv_sql = f"""
    SELECT DISTINCT ON(cusip)
        cusip, int_rate, issue_date, maturity_date,
        price_per100, quantity, int_payment_frequency
    FROM tsy_inventory
    WHERE inventory_date = '{asof.date()}'
    ORDER BY cusip, inventory_date DESC;
    """
    inv = ds.query(inv_sql).to_pandas()
    if inv.empty:
        print(f"No inventory on {asof.date()}")
        return

    # prepare yield curve
    yc = get_yield_curve(asof, ds)
    if yc is None:
        print(f"No curve for {asof.date()}")
        return

    # instantiate bond objects (no price arg)
    bonds = [Bond(r.cusip, r.issue_date, r.maturity_date,
                  r.int_rate, r.int_payment_frequency)
             for r in inv.itertuples()]

    # batch PV calculation
    pvs = Bond.price_batch(bonds, asof, yc)

    # assemble results DataFrame
    results = inv.copy().reset_index(drop=True)
    results['price_closedform'] = pvs
    results['valuation_date'] = asof.date()
    results['time_to_maturity'] = (
        pd.to_datetime(results['maturity_date']) - asof
    ).dt.days / 365.25
    results['coupon'] = results['int_rate'].fillna(0)

    # batch upsert
    values = []
    for row in results.itertuples(index=False):
        values.append(
            f"('{row.cusip}','{row.valuation_date}',{row.price_per100},"
            f"{row.coupon},'{row.maturity_date}',{row.time_to_maturity},"
            f"{row.price_closedform},{row.quantity})"
        )
    vals_sql = ','.join(values)
    upsert_sql = f"""
    INSERT INTO tsy_valuations (
      cusip, valuation_date, entry_price, coupon, maturity_date,
      time_to_maturity, price_closedform, quantity
    ) VALUES {vals_sql}
    ON CONFLICT(cusip, valuation_date) DO UPDATE SET
      entry_price = EXCLUDED.entry_price,
      coupon = EXCLUDED.coupon,
      maturity_date = EXCLUDED.maturity_date,
      time_to_maturity = EXCLUDED.time_to_maturity,
      price_closedform = EXCLUDED.price_closedform,
      quantity = EXCLUDED.quantity,
      updated_at = CURRENT_TIMESTAMP;
    """
    ds.query(upsert_sql)
    print(f"✅ Valued {len(bonds)} bonds on {asof.date()}")

if __name__ == '__main__':
    arg = None
    if len(sys.argv) > 1 and not sys.argv[1].startswith('-'):
        arg = sys.argv[1]
    try:
        pd.to_datetime(arg)
        run_valuation(arg)
    except Exception:
        today = "2025-05-28"
        print(f"Invalid date '{arg}', defaulting to {today}")
        run_valuation(today)


'data_env' not found in environment. Defaulting to 'sandbox' env.
setting env to sandbox data
getting data source for sandbox
Invalid date 'None', defaulting to 2025-05-28
✅ Valued 354 bonds on 2025-05-28
